![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FRetrieval&file=Retrieval+-+Cloud+SQL+For+PostgreSQL.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Cloud%20SQL%20For%20PostgreSQL.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FRetrieval%2FRetrieval%2520-%2520Cloud%2520SQL%2520For%2520PostgreSQL.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Cloud%20SQL%20For%20PostgreSQL.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Retrieval/Retrieval%20-%20Cloud%20SQL%20For%20PostgreSQL.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Retrieval - Cloud SQL For PostgreSQL

In prior workflows, a series of documents was [processed into chunks](../Chunking/readme.md), and for each chunk, [embeddings](../Embeddings/readme.md) were created:

- Process: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb)
- Embed: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

Retrieving chunks for a query involves calculating the embedding for the query and then using similarity metrics to find relevant chunks. A thorough review of similarity matching can be found in [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb) - use dot product! As development moves from experiment to application, the process of storing and computing similarity is migrated to a [retrieval](./readme.md) system. This workflow is part of a [series of workflows exploring many retrieval systems](./readme.md).  

A detailed [comparison of many retrieval systems](./readme.md#comparison-of-vector-database-solutions) can be found in the readme as well.

---

**Cloud SQL For Postgres For Storage, Indexing, And Search**

[Cloud SQL for PostgreSQL](https://cloud.google.com/sql/docs/postgres) is a fully managed relational database service on Google Cloud that offers compatibility with the popular PostgreSQL open-source database. It provides high availability, scalability, and security for your applications.

- **Key Features:**

    - **Fully Managed:** Cloud SQL takes care of database management tasks, including patching, backups, and high availability, allowing you to focus on your applications.
    - **Scalability:** You can easily scale your Cloud SQL instances to accommodate growing data and traffic demands.
    - **Security:** Cloud SQL provides robust security features, including data encryption and network security, to protect your data.
    - **PostgreSQL Compatibility:**  Cloud SQL is compatible with standard PostgreSQL, making it easy to migrate existing applications or use familiar tools and frameworks.

- **Vector Similarity Search with `pgvector`:**

    - Cloud SQL for PostgreSQL supports the [`vector` extension](https://cloud.google.com/sql/docs/postgres/work-with-vectors#required-database-extensions) (based on the open-source [`pgvector` project](https://github.com/pgvector/pgvector)), which adds vector similarity search capabilities to your database.
    - You can [store vector embeddings](https://cloud.google.com/sql/docs/postgres/work-with-vectors#store) directly in your PostgreSQL tables.
    - `vector` provides multiple distance measures (e.g., Euclidean, cosine distance) for [calculating similarity between vectors](https://cloud.google.com/sql/docs/postgres/work-with-vectors#index).
    - You can perform brute-force similarity searches without an index.
    - For efficient approximate nearest neighbor (ANN) search, you can [create indexes](https://cloud.google.com/sql/docs/postgres/work-with-vectors#index) using the IVFFlat and HNSW indexes from `pgvector`.

---

**Use Case Data**

Buying a home usually involves borrowing money from a lending institution, typically through a mortgage secured by the home's value. But how do these institutions manage the risks associated with such large loans, and how are lending standards established?

In the United States, two government-sponsored enterprises (GSEs) play a vital role in the housing market:

- Federal National Mortgage Association ([Fannie Mae](https://www.fanniemae.com/))
- Federal Home Loan Mortgage Corporation ([Freddie Mac](https://www.freddiemac.com/))

These GSEs purchase mortgages from lenders, enabling those lenders to offer more loans. This process also allows Fannie Mae and Freddie Mac to set standards for mortgages, ensuring they are responsible and borrowers are more likely to repay them. This system makes homeownership more affordable and stabilizes the housing market by maintaining a steady flow of liquidity for lenders and keeping interest rates controlled.

However, navigating the complexities of these GSEs and their extensive servicing guides can be challenging.

**Approaches**

[This series](../readme.md) covers many generative AI workflows. These documents are used directly as long context for Gemini in the workflow [Long Context Retrieval With The Vertex AI Gemini API](../Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb). The workflow below uses a [retrieval](./readme.md) approach with the already generated chunks and embeddings.

---
## Colab Setup

When running this notebook in [Colab](https://colab.google/) or [Colab Enterprise](https://cloud.google.com/colab/docs/introduction), this section will authenticate to GCP (follow prompts in the popup) and set the current project for the session.

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Installs and API Enablement

The clients packages may need installing in this environment. 

### Installs (If Needed)

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('google.cloud.sql', 'cloud-sql-python-connector'),
    ('sqlalchemy', 'sqlalchemy'),
    ('pg8000', 'pg8000'),
    ('asyncpg', 'asyncpg')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable sqladmin.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'retrieval-cloudsql-postgresql'

# Cloud SQL Names
CLOUDSQL_INSTANCE_NAME = EXPERIMENT
CLOUDSQL_DATABASE_NAME = SERIES
CLOUDSQL_TABLE_NAME = EXPERIMENT

CLOUDSQL_USER = 'test_db'
CLOUDSQL_PASS = 'test_db_pass'

Packages

In [8]:
import os, json, time, glob, datetime, asyncio

import numpy as np

# Vertex AI
from google.cloud import aiplatform
import vertexai.language_models # for embeddings API
import vertexai.generative_models # for Gemini Models
from vertexai.resources.preview import feature_store

# Cloud SQL
import google.cloud.sql.connector
import pg8000
import sqlalchemy
import asyncpg
import sqlalchemy.ext.asyncio

In [9]:
aiplatform.__version__

'1.71.0'

Clients

In [10]:
# vertex ai clients
vertexai.init(project = PROJECT_ID, location = REGION)

---
## Text & Embeddings For Examples

This repository contains a [section for document processing (chunking)](../Chunking/readme.md) that includes an example of processing mulitple large pdfs (over 1000 pages) into chunks: [Large Document Processing - Document AI Layout Parser](../Chunking/Large%20Document%20Processing%20-%20Document%20AI%20Layout%20Parser.ipynb).  The chunks of text from that workflow are stored with this repository and loaded by another companion workflow that augments the chunks with text embeddings: [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb).

The following code will load the version of the chunks that includes text embeddings and prepare it for a local example of retrival augmented generation.

### Get The Documents

If you are working from a clone of this notebooks [repository](https://github.com/statmike/vertex-ai-mlops) then the documents are already present. The following cell checks for the documents folder and if it is missing gets it (`git clone`):

In [11]:
local_dir = '../Embeddings/files/embeddings-api'

In [12]:
if not os.path.exists(local_dir):
    print('Retrieving documents...')
    parent_dir = os.path.dirname(local_dir)
    temp_dir = os.path.join(parent_dir, 'temp')
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    !git clone https://www.github.com/statmike/vertex-ai-mlops {temp_dir}/vertex-ai-mlops
    shutil.copytree(f'{temp_dir}/vertex-ai-mlops/Applied GenAI/Embeddings/files/embeddings-api', local_dir)
    shutil.rmtree(temp_dir)
    print(f'Documents are now in folder `{local_dir}`')
else:
    print(f'Documents Found in folder `{local_dir}`')             

Documents Found in folder `../Embeddings/files/embeddings-api`


### Load The Chunks

In [13]:
jsonl_files = glob.glob(f"{local_dir}/large-files*.jsonl")
jsonl_files.sort()
jsonl_files

['../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0000.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0001.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0002.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0003.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0004.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0005.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0006.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0007.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0008.jsonl',
 '../Embeddings/files/embeddings-api/large-files-chunk-embeddings-0009.jsonl']

In [14]:
chunks = []
for file in jsonl_files:
    with open(file, 'r') as f:
        chunks.extend([json.loads(line) for line in f])
len(chunks)

9040

### Review A Chunk

In [15]:
chunks[0].keys()

dict_keys(['instance', 'predictions', 'status'])

In [16]:
chunks[0]['instance']['chunk_id']

'fannie_part_0_c17'

In [17]:
print(chunks[0]['instance']['content'])

# Selling Guide Fannie Mae Single Family

## Fannie Mae Copyright Notice

### Fannie Mae Copyright Notice

|-|
| Section B3-4.2, Verification of Depository Assets 402 |
| B3-4.2-01, Verification of Deposits and Assets (05/04/2022) 403 |
| B3-4.2-02, Depository Accounts (12/14/2022) 405 |
| B3-4.2-03, Individual Development Accounts (02/06/2019) 408 |
| B3-4.2-04, Pooled Savings (Community Savings Funds) (04/01/2009) 411 |
| B3-4.2-05, Foreign Assets (05/04/2022) 411 |
| Section B3-4.3, Verification of Non-Depository Assets 412 |
| B3-4.3-01, Stocks, Stock Options, Bonds, and Mutual Funds (06/30/2015) 412 |
| B3-4.3-02, Trust Accounts (04/01/2009) 413 |
| B3-4.3-03, Retirement Accounts (06/30/2015) 414 |
| B3-4.3-04, Personal Gifts (09/06/2023) 415 |
| B3-4.3-05, Gifts of Equity (10/07/2020) 418 |
| B3-4.3-06, Grants and Lender Contributions (12/14/2022) 419 |
| B3-4.3-07, Disaster Relief Grants or Loans (04/01/2009) 423 |
| B3-4.3-08, Employer Assistance (09/29/2015) 423 |
| B3-4.3-09,

In [18]:
chunks[0]['predictions'][0]['embeddings']['values'][0:10]

[0.031277116388082504,
 0.03056905046105385,
 0.010865348391234875,
 0.0623614676296711,
 0.03228681534528732,
 0.05066155269742012,
 0.046544693410396576,
 0.05509665608406067,
 -0.014074751175940037,
 0.008380400016903877]

### Prepare Chunk Structure

Make a list of dictionaries with information for each chunk:

In [19]:
content_chunks = [
    dict(
        gse = chunk['instance']['gse'],
        chunk_id = chunk['instance']['chunk_id'],
        content = chunk['instance']['content'],
        embedding = chunk['predictions'][0]['embeddings']['values']
    ) for chunk in chunks
]

### Query Embedding

Create a query, or prompt, and get the embedding for it:

Connect to models for text embeddings. Learn more about the model API:
- [Vertex AI Text Embeddings API](../Embeddings/Vertex%20AI%20Text%20Embeddings%20API.ipynb)

In [11]:
question = "Does a lender have to perform servicing functions directly?"

In [12]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')

In [13]:
question_embedding = embedder.get_embeddings([question])[0].values
question_embedding[0:10]

[-0.0005117303808219731,
 0.009651427157223225,
 0.01768726110458374,
 0.014538003131747246,
 -0.01829824410378933,
 0.027877431362867355,
 -0.021124685183167458,
 0.008830446749925613,
 -0.02669006586074829,
 0.06414774805307388]

---
## Setup Cloud SQL For PostgreSQL

Cloud SQL For PostgreSQL is a managed instance of [PostgreSQL](https://www.postgresql.org/) that is easy to deploy and use.  This workflow will create an instance, configure it, load and use data all in the workflow below. While failover replicas and read-only replicas are possible this example uses a single instance server with minimum possible specifications.  The choices made during creation of instances fall into categories called [editions](https://cloud.google.com/sql/docs/postgres/editions-intro).  The configuration choices are also related to overall cost of running the instance - [see pricing](https://cloud.google.com/sql/pricing#mysql-pg-pricing).  This workflow uses a minimal configuration for testing purposes and keeping the cost of this example very small.  At the end of this notebook is a section that can be used to shutdown and delete the parts related to ongoing costs.

### Create/Retrieve Instance

The starting point for using Cloud SQL for MySQL is creating an instance.  There is not a Python admin client for Cloud SQL so the `gcloud` cli is used from this notebook by prefixing them with `!` to indicate they are to be run as shell commands on the underlying system shell.

The documentation can be referenced for:
- [Create instances](https://cloud.google.com/sql/docs/postgres/create-instance)
- [Instance settings](https://cloud.google.com/sql/docs/postgres/instance-settings)
- [Cloud SDK (gcloud) CLI For SQL instances](https://cloud.google.com/sdk/gcloud/reference/sql/instances)

In [14]:
list_instances = !gcloud sql instances list --format="json(name)"
list_instances = json.loads(''.join(list_instances))

if CLOUDSQL_INSTANCE_NAME in [i['name'] for i in list_instances]:
    instance_describe = !gcloud sql instances describe $CLOUDSQL_INSTANCE_NAME --format=json
    instance_describe = json.loads(''.join(instance_describe))
    print(f"Found the instance: {instance_describe['name']}")
else:
    print('Creating an instance...')
    instance_create = !gcloud sql instances create $CLOUDSQL_INSTANCE_NAME \
        --database-version=POSTGRES_15 \
        --tier=db-g1-small \
        --region=us-central1 \
        --quiet
    instance_describe = !gcloud sql instances describe $CLOUDSQL_INSTANCE_NAME --format=json
    instance_describe = json.loads(''.join(instance_describe))
    print(f"Created the instance: {instance_describe['name']}")

Found the instance: retrieval-cloudsql-postgresql


In [45]:
#instance_describe

### Create User For This Workflow

We need a user account to login and use the PostgreSQL instance.  In production this should be taken very seriously and access should be configured carefully to protect the environment.  This example create an example user with password for demonstration and testing.  Read more about controlling access in [About PostgreSQL users](https://cloud.google.com/sql/docs/postgres/users).

**Reference:**
- [Cloud SDK (gcloud) CLI for SQL users](https://cloud.google.com/sql/docs/mysql/users)

In [16]:
list_users = !gcloud sql users list --format="json" --instance=$CLOUDSQL_INSTANCE_NAME
list_users = json.loads(''.join(list_users))

if CLOUDSQL_USER in [i['name'] for i in list_users]:
    user_describe = !gcloud sql users describe $CLOUDSQL_USER --instance=$CLOUDSQL_INSTANCE_NAME --format=json
    user_describe = json.loads(''.join(user_describe))
    print(f"Found the user: {user_describe['name']}")
else:
    print('Creating the user...')
    user_create = !gcloud sql users create $CLOUDSQL_USER \
        --instance=$CLOUDSQL_INSTANCE_NAME \
        --password=$CLOUDSQL_PASS
    user_describe = !gcloud sql users describe $CLOUDSQL_USER --instance=$CLOUDSQL_INSTANCE_NAME --format=json
    user_describe = json.loads(''.join(user_describe))
    print(f"Created the user: {user_describe['name']}")

Found the user: test_db


In [17]:
#user_describe

### Connection To Databases

Cloud SQL For PostgreSQL has a default database, [postgres](https://cloud.google.com/sql/docs/postgres/create-manage-databases).

There are many ways to [connect to a database](https://cloud.google.com/sql/docs/postgres/connect-overview) depending on where and how you need to connect.

Here we want to use Python and will use the convenient [Cloud SQL Language Connectors](https://cloud.google.com/sql/docs/postgres/language-connectors).

That means we will [create a connector](https://cloud.google.com/sql/docs/postgres/connect-connectors#python) and interact with the database through the connector.  A connector has three parts:
- a **connection tool**, in this case provided by [Cloud SQL Language Connectors](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector)
- drivers to create a **connection pool**
    - synchronous with [pg8000](https://github.com/tlocke/pg8000)
    - asynchronous with [asyncpg](https://github.com/MagicStack/asyncpg)
- a client library that can use connection pools to **orchestrate SQL queries**, [SQLAlchemy](https://www.sqlalchemy.org/)

#### Connection Tool

In [18]:
sync_connector = google.cloud.sql.connector.Connector()
async_connector = await google.cloud.sql.connector.create_async_connector()

#### Connection

In [19]:
def get_sync_conn(
    connector: google.cloud.sql.connector.Connector,
    db: str
):
    def getconn():
        conn = connector.connect(
            instance_describe['connectionName'],
            "pg8000",
            user = CLOUDSQL_USER,
            password = CLOUDSQL_PASS,
            db = db
        )
        return conn
    return getconn

In [20]:
async def get_async_conn(
    connector: google.cloud.sql.connector.Connector,
    db: str
):
    async def getconn():
        conn = await connector.connect_async(
            instance_describe['connectionName'],
            "asyncpg",
            user = CLOUDSQL_USER,
            password = CLOUDSQL_PASS,
            db = db
        )
        return conn
    return getconn

#### Connection Pool

In [21]:
def get_sync_pool(
    connector: google.cloud.sql.connector.Connector,
    db: str
) -> sqlalchemy.engine.Engine:

    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator = get_sync_conn(connector, db)
    )
    pool.dialect.description_encoding = None
    pool.execution_options(isolation_level="AUTOCOMMIT")
    return pool

In [22]:
async def get_async_pool(
    connector: google.cloud.sql.connector.Connector,
    db: str
) -> sqlalchemy.engine.Engine:

    pool = sqlalchemy.ext.asyncio.create_async_engine(
        "postgresql+asyncpg://",
        async_creator = await get_async_conn(connector, db)
    )
    pool.dialect.description_encoding = None
    pool.execution_options(isolation_level="AUTOCOMMIT")
    return pool

In [23]:
sync_pool = get_sync_pool(sync_connector, 'postgres')

In [24]:
async_pool = await get_async_pool(async_connector, 'postgres')

#### Query Orchestrator

Use the a pool as a context manager to orchstrate queries

In [25]:
def run_query(query, pool = None, connector = sync_connector):
    # get the current connnection pool:
    if pool is None:
        pool = sync_pool
        
    # run the query and get the response as 'result'
    with pool.connect().execution_options(isolation_level="AUTOCOMMIT") as connection:
        result = connection.execute(query)
        #connector.close()
        
    # prepare the response
    rows = []
    try:
        for row in result:
            rows.append(dict(zip(result.keys(), row)))
    except Exception:
        pass
    
    # return the response
    return rows[0] if len(rows) == 1 else rows

In [26]:
async def async_run_query(query, pool = None, connector = async_connector):
    # get the current connection pool
    if pool is None:
        pool = async_pool
        
    # run the query and get the response as 'result'
    async with pool.connect() as connection:
        result = await connection.execute(query)
        await connection.commit()
        #await connector.close_async()
        
    # prepare the response
    rows = []
    try:
        for row in result:
            rows.append(dict(zip(result.keys(), row)))
    except Exception:
        pass
    
    # return the response
    return rows[0] if len(rows) == 1 else rows

#### Execute A Test Query

When submitting SQL statements either of the connectors should work for DML (SELECT, INSERT, DELETE, UPDATE) but the synchronous connector should be preferred for DDL (CREATE, ALTER, DROP) statements.

In [27]:
query = sqlalchemy.text("SELECT 'Success' as did_it_work")

In [28]:
run_query(query)

{'did_it_work': 'Success'}

#### Execute Async Queries

Use [`asyncio`](https://docs.python.org/3/library/asyncio.html) to work with async queries.

In [29]:
await async_run_query(query)

{'did_it_work': 'Success'}

In [30]:
queries = [query]*5
tasks = [async_run_query(q) for q in queries]
results = await asyncio.gather(*tasks)
results

[{'did_it_work': 'Success'},
 {'did_it_work': 'Success'},
 {'did_it_work': 'Success'},
 {'did_it_work': 'Success'},
 {'did_it_work': 'Success'}]

---
## Working With Cloud SQL For PostgreSQL

Now that a connection to PostgreSQL is established the environment can be interacted with using, SQL!

### Create A Database

**PostgreSQL References:**
- [CREATE DATABASE](https://www.postgresql.org/docs/current/sql-createdatabase.html)
- [The catalog `pg_database`](https://www.postgresql.org/docs/current/catalog-pg-database.html)

In [43]:
query = sqlalchemy.text(f"SELECT datname FROM pg_database WHERE datname = '{CLOUDSQL_DATABASE_NAME}'")
result = run_query(query)
result

[]

In [44]:
if not result:
    query = sqlalchemy.text(f"CREATE DATABASE \"{CLOUDSQL_DATABASE_NAME}\"")
    run_query(query)

In [45]:
query = sqlalchemy.text(f"SELECT * FROM pg_database WHERE datname = '{CLOUDSQL_DATABASE_NAME}'")
run_query(query)

{'oid': 53059,
 'datname': 'applied-genai',
 'datdba': 16538,
 'encoding': 6,
 'datlocprovider': 'c',
 'datistemplate': False,
 'datallowconn': True,
 'datconnlimit': -1,
 'datfrozenxid': 716,
 'datminmxid': 1,
 'dattablespace': 1663,
 'datcollate': 'en_US.UTF8',
 'datctype': 'en_US.UTF8',
 'daticulocale': None,
 'datcollversion': '2.19',
 'datacl': None}

### Move Connection To New Database

Note that the connection pool connects to a specific database.  Now that a new database is created we can switch the connection pool to it by first closing the existing connection pool and creating a new one.

Verify database for current connection:

In [46]:
run_query(sqlalchemy.text('SELECT current_database()'))

{'current_database': 'postgres'}

In [47]:
await async_run_query(sqlalchemy.text('SELECT current_database()'))

{'current_database': 'postgres'}

Close the current connection and create a new one:

In [31]:
sync_pool.dispose()
sync_connector.close()
sync_connector = google.cloud.sql.connector.Connector()
sync_pool = get_sync_pool(sync_connector, CLOUDSQL_DATABASE_NAME)

await async_pool.dispose()
await async_connector.close_async()
async_connector = await google.cloud.sql.connector.create_async_connector()
async_pool = await get_async_pool(async_connector, CLOUDSQL_DATABASE_NAME)

Verify the database of the new connection:

In [49]:
run_query(sqlalchemy.text('SELECT current_database()'))

{'current_database': 'applied-genai'}

In [50]:
await async_run_query(sqlalchemy.text('SELECT current_database()'))

{'current_database': 'applied-genai'}

### Create Table


**PostgreSQL References:**
- [CREATE TABLE statement](https://www.postgresql.org/docs/current/sql-createtable.html)
- [Information Schema Tables](https://www.postgresql.org/docs/current/information-schema.html)

In [51]:
result = run_query(sqlalchemy.text(f"SELECT * from information_schema.tables WHERE table_name = '{CLOUDSQL_TABLE_NAME}'"))
result

[]

In [52]:
run_query(sqlalchemy.text(f"DROP TABLE IF EXISTS \"{CLOUDSQL_TABLE_NAME}\""))

[]

In [53]:
run_query(
    sqlalchemy.text(f"""
            CREATE TABLE IF NOT EXISTS \"{CLOUDSQL_TABLE_NAME}\" (
                chunk_id VARCHAR(100) NOT NULL PRIMARY KEY,
                gse VARCHAR(50),
                content TEXT,
                embedding REAL[]
            );
        """
    )
)

[]

In [54]:
result = run_query(sqlalchemy.text(f"SELECT * from information_schema.tables WHERE table_name = '{CLOUDSQL_TABLE_NAME}'"))
result

{'table_catalog': 'applied-genai',
 'table_schema': 'public',
 'table_name': 'retrieval-cloudsql-postgresql',
 'table_type': 'BASE TABLE',
 'self_referencing_column_name': None,
 'reference_generation': None,
 'user_defined_type_catalog': None,
 'user_defined_type_schema': None,
 'user_defined_type_name': None,
 'is_insertable_into': 'YES',
 'is_typed': 'NO',
 'commit_action': None}

In [55]:
run_query(sqlalchemy.text(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{CLOUDSQL_TABLE_NAME}'"))

[{'column_name': 'embedding', 'data_type': 'ARRAY'},
 {'column_name': 'chunk_id', 'data_type': 'character varying'},
 {'column_name': 'gse', 'data_type': 'character varying'},
 {'column_name': 'content', 'data_type': 'text'}]

### Add, Retrieve, And Delete Rows

Learn about inserting, retrieving and deleting records/rows with the following simple examples.

#### Get A Record

Dictionaries for each record/row are stored in `content_chunks` from earlier in this workflow:

In [56]:
first_record = content_chunks[0]

In [57]:
first_record.keys()

dict_keys(['gse', 'chunk_id', 'content', 'embedding'])

In [58]:
first_record['chunk_id']

'fannie_part_0_c17'

#### Insert Row

In [59]:
table = sqlalchemy.Table(
    CLOUDSQL_TABLE_NAME,
    sqlalchemy.MetaData(),
    autoload_with = sync_pool
)

In [60]:
for c in table.columns:
    print(c)

retrieval-cloudsql-postgresql.chunk_id
retrieval-cloudsql-postgresql.gse
retrieval-cloudsql-postgresql.content
retrieval-cloudsql-postgresql.embedding


In [61]:
insert_row = sqlalchemy.insert(table).values(first_record)

In [62]:
run_query(insert_row)

[]

#### Retrieve Row

There are two helpful ways to retrieve rows.  Both with SQL and with the sqlalchemy clients `select` method.  Both are demonstrated here.

Using SQL:

In [63]:
query = sqlalchemy.text(f"SELECT * FROM \"{CLOUDSQL_TABLE_NAME}\" WHERE chunk_id = '{first_record['chunk_id']}'")
result = run_query(query)

In [64]:
result.keys()

dict_keys(['chunk_id', 'gse', 'content', 'embedding'])

In [65]:
result['chunk_id']

'fannie_part_0_c17'

Using sqlalchemy clients `select` method:

In [66]:
query = sqlalchemy.select(table).where(table.columns.chunk_id == first_record['chunk_id'])
result = run_query(query)

In [67]:
result.keys()

dict_keys(['chunk_id', 'gse', 'content', 'embedding'])

In [68]:
result['chunk_id']

'fannie_part_0_c17'

In [69]:
type(result['embedding'])

list

In [70]:
result['embedding'][0:10]

[0.031277116,
 0.03056905,
 0.010865348,
 0.062361468,
 0.032286815,
 0.050661553,
 0.046544693,
 0.055096656,
 -0.014074751,
 0.0083804]

#### Delete Row

Delete the row added here.  Verify the action by counting the rows before and after the deletion.

In [71]:
run_query(sqlalchemy.text(f"SELECT COUNT(*) as count FROM \"{CLOUDSQL_TABLE_NAME}\""))

{'count': 1}

In [72]:
run_query(sqlalchemy.text(f"DELETE FROM \"{CLOUDSQL_TABLE_NAME}\" WHERE chunk_id = '{first_record['chunk_id']}'"))

[]

In [73]:
run_query(sqlalchemy.text(f"SELECT COUNT(*) as count FROM \"{CLOUDSQL_TABLE_NAME}\""))

{'count': 0}

## Load Data 

There are a lot of rows to load but [using `asyncio`](https://docs.python.org/3/library/asyncio.html) with the async connection makes this easy to orchestrate:

Create a list of query statements:

In [74]:
queries = [sqlalchemy.insert(table).values(c) for c in content_chunks]

Create a list of task that will run the queries.  Do not `await` these yet.

In [75]:
tasks = [async_run_query(query) for query in queries]

Run all the tasks with `asyncio.gather` and await the result.

In [76]:
results = await asyncio.gather(*tasks)

Verify the results with a row count:

In [77]:
run_query(sqlalchemy.text(f"SELECT COUNT(*) as count FROM \"{CLOUDSQL_TABLE_NAME}\""))

{'count': 9040}

---
## Setup Cloud SQL For PostgreSQL For Vector Similarity Search

[Working with vector embeddings](https://cloud.google.com/sql/docs/postgres/work-with-vectors) requires [database extentions](https://cloud.google.com/sql/docs/postgres/work-with-vectors#required-database-extensions) to support vectors.

**Store Embeddings As Vectors**

[Google provides](https://cloud.google.com/sql/docs/postgres/work-with-vectors#required-database-extensions) a version of [`pgvector`](https://github.com/pgvector/pgvector#indexing) named `vector` that includes functions and operators for working with vector values. 

```CREATE EXTENSION IF NOT EXISTS vector```

In [78]:
run_query(sqlalchemy.text(f"CREATE EXTENSION IF NOT EXISTS vector"))

[]

### Convert `embedding` Column To Vector Data Type

The data was loaded/inserted above with the embedding stored in a column named 'embedding' as an ARRAY of float values.  This column can now be converted to the vector type with the specific dimension using an `ALTER TABLE` command.

In [79]:
run_query(sqlalchemy.text(f"ALTER TABLE \"{CLOUDSQL_TABLE_NAME}\" ALTER COLUMN embedding TYPE vector({len(question_embedding)});"))

[]

In [80]:
run_query(sqlalchemy.text(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{CLOUDSQL_TABLE_NAME}'"))

[{'column_name': 'embedding', 'data_type': 'USER-DEFINED'},
 {'column_name': 'chunk_id', 'data_type': 'character varying'},
 {'column_name': 'gse', 'data_type': 'character varying'},
 {'column_name': 'content', 'data_type': 'text'}]

In [81]:
query = sqlalchemy.text(f"SELECT * FROM \"{CLOUDSQL_TABLE_NAME}\" WHERE chunk_id = '{first_record['chunk_id']}'")
result = run_query(query)
result.keys()

dict_keys(['chunk_id', 'gse', 'content', 'embedding'])

In [82]:
type(result['embedding'])

str

In [83]:
result['embedding'][0:100]

'[0.031277116,0.03056905,0.010865348,0.062361468,0.032286815,0.050661553,0.046544693,0.055096656,-0.0'

---
## Vector Similarity Search, Matching

This section covers the operation of using a vector similarity metric calculation to find nearest neighbors for a query vector while also taking advantage of indexing.  To understand similarity metrics and motivate the intution for choosing one (choose dot product), check out [The Math of Similarity](../Embeddings/The%20Math%20of%20Similarity.ipynb).

### Check For Vector Indexes

At this point in the workflow no vector indexes have been created.  The following cells show how to check for indexes and will be reused later in the workflow to verify the details of indexes after they are created.

In [84]:
run_query(sqlalchemy.text(f"""
    SELECT *
    FROM pg_indexes 
    WHERE tablename = '{CLOUDSQL_TABLE_NAME}'
"""))

{'schemaname': 'public',
 'tablename': 'retrieval-cloudsql-postgresql',
 'indexname': 'retrieval-cloudsql-postgresql_pkey',
 'tablespace': None,
 'indexdef': 'CREATE UNIQUE INDEX "retrieval-cloudsql-postgresql_pkey" ON public."retrieval-cloudsql-postgresql" USING btree (chunk_id)'}

### Brute Force Search - No Index

Without an index you can still use distance measures to find nearest neighbor matches through brute force search that compare a query embedding to all rows.

Easily run a brute force (compare to all rows) match with a choice of distance measure using the [`pgvector` querying notation](https://github.com/pgvector/pgvector?tab=readme-ov-file#querying):
- `<=>` for Cosine distance
- `<->` for L2, Euclidean distance
- `<#>` for Dot product
    - this is actually the negative of the inner product

Dot product with `<#>`

In [85]:
run_query(sqlalchemy.text(f"""
    SELECT
        chunk_id,
        embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547}]

Euclidean distance with `<->`

In [133]:
run_query(sqlalchemy.text(f"""
    SELECT
        chunk_id,
        embedding <-> '{question_embedding}' AS euclidean_distance
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY euclidean_distance
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'euclidean_distance': 0.7615658337594855},
 {'chunk_id': 'freddie_part_4_c509', 'euclidean_distance': 0.7992875100367289},
 {'chunk_id': 'freddie_part_4_c510', 'euclidean_distance': 0.8057848660615564},
 {'chunk_id': 'fannie_part_0_c353', 'euclidean_distance': 0.8094337265330812},
 {'chunk_id': 'fannie_part_0_c326', 'euclidean_distance': 0.8144253147417732}]

Cosine Similarity with `<=>`

In [137]:
run_query(sqlalchemy.text(f"""
    SELECT
        chunk_id,
        embedding <=> '{question_embedding}' AS cosine_similarity
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY cosine_similarity
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'cosine_similarity': 0.2899983636254655},
 {'chunk_id': 'freddie_part_4_c509', 'cosine_similarity': 0.31944424887732137},
 {'chunk_id': 'freddie_part_4_c510', 'cosine_similarity': 0.3246529458452222},
 {'chunk_id': 'fannie_part_0_c353', 'cosine_similarity': 0.32760391792511945},
 {'chunk_id': 'fannie_part_0_c326', 'cosine_similarity': 0.33164633285935186}]

### Brute Force Search With Pre-Filtering - No Index

Extending a brute force match with pre-filtering means including a `WHERE` statement to first filter to row that meet a desired condition:

Find the top 5 matches where the GSE is 'fannie':

In [91]:
run_query(sqlalchemy.text(f"""
    SELECT
        chunk_id,
        embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    WHERE gse = 'fannie'
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547},
 {'chunk_id': 'fannie_part_0_c92', 'dot_product': -0.6614338159561157},
 {'chunk_id': 'fannie_part_0_c240', 'dot_product': -0.6608578562736511}]

Find the top 5 matches where the GSE is 'freddie':

In [92]:
run_query(sqlalchemy.text(f"""
    SELECT
        chunk_id,
        embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    WHERE gse = 'freddie'
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'freddie_part_4_c472', 'dot_product': -0.661984384059906},
 {'chunk_id': 'freddie_part_6_c439', 'dot_product': -0.6604534983634949},
 {'chunk_id': 'freddie_part_4_c558', 'dot_product': -0.6575403213500977}]

### Create And Use An Index

Indexes make search across many rows more efficient by first matching partions of rows and then only comparing to rows within the partions.  This section covers [creating indexes](https://cloud.google.com/sql/docs/postgres/work-with-vectors#index) and using them in queries.

- IVFFlat: Inverted File Lists, specifically with flat quantization
    - partions rows into list, only searches a subset that are closest to the query vector
    - fast build, low memory usage, slower query
    - can increase the number of lists used in search at query time for greater recall
    - Reference [pgvector Indexing IVFFlat](https://github.com/pgvector/pgvector?tab=readme-ov-file#ivfflat)
- [HNSW](https://arxiv.org/abs/1603.09320): Hierarchical Navigable Small World graphs
    - creates a multilayer graph
    - slower build, more memory, faster query
    - can increase the number of candidates in the search for greater recall
    - Reference [pgvector Indexing HNSW](https://github.com/pgvector/pgvector?tab=readme-ov-file#hnsw)
    
The query optimizer will use indexes to speed up queries.  If multiple indexes are present on the embedding column then the optimizer will select the best for the query.  Some queries may also trigger full row scans, brute force, matching.  The example below use the [PosgreSQL `EXPLAIN ANALYZE`](https://www.postgresql.org/docs/current/sql-explain.html) to understand the impact of index on differen type of matching queries.

#### Index: `IVFFlat`

Reference [pgvector Indexing IVFFlat](https://github.com/pgvector/pgvector?tab=readme-ov-file#ivfflat)

Create the index:

In [93]:
run_query(sqlalchemy.text(f"""
    CREATE INDEX IF NOT EXISTS ivfflat_index
    ON \"{CLOUDSQL_TABLE_NAME}\"
    USING ivfflat (embedding vector_ip_ops)
    WITH (lists = 100)
"""))

[]

Review the index details:

In [94]:
run_query(sqlalchemy.text(f"SELECT * FROM pg_indexes  WHERE tablename = '{CLOUDSQL_TABLE_NAME}' AND indexname = 'ivfflat_index'"))

{'schemaname': 'public',
 'tablename': 'retrieval-cloudsql-postgresql',
 'indexname': 'ivfflat_index',
 'tablespace': None,
 'indexdef': 'CREATE INDEX ivfflat_index ON public."retrieval-cloudsql-postgresql" USING ivfflat (embedding vector_ip_ops) WITH (lists=\'100\')'}

In [95]:
run_query(sqlalchemy.text('SELECT * FROM pg_stat_progress_create_index'))

[]

Use distance measure, dot product with `<#>`, with the index:

In [96]:
run_query(sqlalchemy.text(f"""
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'freddie_part_5_c360', 'dot_product': -0.6557323932647705},
 {'chunk_id': 'freddie_part_4_c508', 'dot_product': -0.6519314646720886},
 {'chunk_id': 'freddie_part_5_c290', 'dot_product': -0.6456407904624939}]

Use `EXPLAIN ANALYZE` to understand the query execution.  Note that the index was used:

In [97]:
result = run_query(sqlalchemy.text(f"""
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=115.68..115.88 rows=5 width=27) (actual time=0.388..0.429 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using ivfflat_index on "retrieval-cloudsql-postgresql"  (cost=115.68..487.18 rows=9040 width=27) (actual time=0.387..0.427 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.164 ms'},
 {'QUERY PLAN': 'Execution Time: 0.499 ms'}]

Use the query option `ivfflat.probes = 10` to specify the number of partitions to scan:

In [98]:
run_query(sqlalchemy.text(f"""
SET LOCAL ivfflat.probes = 10;
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547}]

Use `EXPLAIN ANALYZE` to understand the impact of the query option and notice the longer exectuion time:

In [99]:
result = run_query(sqlalchemy.text(f"""
SET LOCAL ivfflat.probes = 10;
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=1156.78..1158.24 rows=5 width=27) (actual time=1.637..1.682 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using ivfflat_index on "retrieval-cloudsql-postgresql"  (cost=1156.78..3792.42 rows=9040 width=27) (actual time=1.636..1.680 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.152 ms'},
 {'QUERY PLAN': 'Execution Time: 1.714 ms'}]

Add a filter, `gse = 'fannie'`, to the query and note that it still returns the request number of matches.  This is pre-filtering.

In [97]:
run_query(sqlalchemy.text(f"""
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    WHERE gse = 'fannie'
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_2_c793', 'dot_product': -0.6455321311950684},
 {'chunk_id': 'fannie_part_2_c795', 'dot_product': -0.6292608976364136},
 {'chunk_id': 'fannie_part_2_c792', 'dot_product': -0.6290532350540161},
 {'chunk_id': 'fannie_part_2_c791', 'dot_product': -0.6204110383987427},
 {'chunk_id': 'fannie_part_2_c798', 'dot_product': -0.6203723549842834}]

Use `EXPLAIN ANALYZE` to see if the index is still used in the pre-filtering query.  Note that it is!

In [100]:
result = run_query(sqlalchemy.text(f"""
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    WHERE gse = 'fannie'
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=115.68..116.27 rows=5 width=27) (actual time=0.373..0.428 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using ivfflat_index on "retrieval-cloudsql-postgresql"  (cost=115.68..472.38 rows=3032 width=27) (actual time=0.371..0.425 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.157 ms'},
 {'QUERY PLAN': 'Execution Time: 0.464 ms'}]

Drop the index:

In [101]:
run_query(sqlalchemy.text('DROP INDEX IF EXISTS ivfflat_index'))

[]

#### Index: `HNSW`

Reference [pgvector Indexing HNSW](https://github.com/pgvector/pgvector?tab=readme-ov-file#hnsw)

Create the index:

In [103]:
run_query(sqlalchemy.text(f"""
    CREATE INDEX IF NOT EXISTS hnsw_index
    ON \"{CLOUDSQL_TABLE_NAME}\"
    USING hnsw (embedding vector_ip_ops)
    WITH (m = 10, ef_construction = 40)
"""))

[]

Review the index details:

In [104]:
run_query(sqlalchemy.text(f"SELECT * FROM pg_indexes  WHERE tablename = '{CLOUDSQL_TABLE_NAME}' AND indexname = 'hnsw_index'"))

{'schemaname': 'public',
 'tablename': 'retrieval-cloudsql-postgresql',
 'indexname': 'hnsw_index',
 'tablespace': None,
 'indexdef': 'CREATE INDEX hnsw_index ON public."retrieval-cloudsql-postgresql" USING hnsw (embedding vector_ip_ops) WITH (m=\'10\', ef_construction=\'40\')'}

In [105]:
run_query(sqlalchemy.text('SELECT * FROM pg_stat_progress_create_index'))

[]

Use distance measure, dot product with `<#>`, with the index:

In [106]:
run_query(sqlalchemy.text(f"""
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547}]

Use `EXPLAIN ANALYZE` to understand the query execution.  Note that the index was used:

In [107]:
result = run_query(sqlalchemy.text(f"""
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=100.38..103.00 rows=5 width=27) (actual time=1.185..1.252 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using hnsw_index on "retrieval-cloudsql-postgresql"  (cost=100.38..4845.38 rows=9040 width=27) (actual time=1.183..1.248 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.136 ms'},
 {'QUERY PLAN': 'Execution Time: 1.289 ms'}]

Use the query option `hnsw.ef_search = 20` to specify the number of partitions to scan:

In [108]:
run_query(sqlalchemy.text(f"""
SET LOCAL hnsw.ef_search = 20;
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547}]

Use `EXPLAIN ANALYZE` to understand the impact of the query option and notice the shorter execution time:

In [110]:
result = run_query(sqlalchemy.text(f"""
SET LOCAL hnsw.ef_search = 20;
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=100.38..103.00 rows=5 width=27) (actual time=0.767..0.809 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using hnsw_index on "retrieval-cloudsql-postgresql"  (cost=100.38..4845.38 rows=9040 width=27) (actual time=0.766..0.807 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.128 ms'},
 {'QUERY PLAN': 'Execution Time: 0.837 ms'}]

Add a filter, `gse = 'fannie'`, to the query and note that it still returns the request number of matches.  This is pre-filtering.

In [111]:
run_query(sqlalchemy.text(f"""
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    WHERE gse = 'fannie'
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547},
 {'chunk_id': 'fannie_part_0_c92', 'dot_product': -0.6614338159561157},
 {'chunk_id': 'fannie_part_0_c240', 'dot_product': -0.6608578562736511}]

Use `EXPLAIN ANALYZE` to see if the index is still used in the pre-filtering query.  Note that it is!

In [112]:
result = run_query(sqlalchemy.text(f"""
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    WHERE gse = 'fannie'
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=100.38..108.21 rows=5 width=27) (actual time=1.090..1.135 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using hnsw_index on "retrieval-cloudsql-postgresql"  (cost=100.38..4852.95 rows=3032 width=27) (actual time=1.089..1.132 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.124 ms'},
 {'QUERY PLAN': 'Execution Time: 1.164 ms'}]

Drop the index:

In [113]:
run_query(sqlalchemy.text('DROP INDEX IF EXISTS hnsw_index'))

[]

### Queries With Multiple Indexes

The optimizer will choose wheather to use an index or not and when multiple indexes are present it will select the most appliable to the query.  This example recreats both IVFFlat and HNSW indexes and run various queries to examine the choices the optimizer makes.

Create the indexes:

In [114]:
run_query(sqlalchemy.text(f"""
    CREATE INDEX IF NOT EXISTS ivfflat_index
    ON \"{CLOUDSQL_TABLE_NAME}\"
    USING ivfflat (embedding vector_ip_ops)
    WITH (lists = 100)
"""))

[]

In [115]:
run_query(sqlalchemy.text(f"""
    CREATE INDEX IF NOT EXISTS hnsw_index
    ON \"{CLOUDSQL_TABLE_NAME}\"
    USING hnsw (embedding vector_ip_ops)
    WITH (m = 10, ef_construction = 40)
"""))

[]

Review the index details:

In [117]:
run_query(sqlalchemy.text(f"SELECT * FROM pg_indexes  WHERE tablename = '{CLOUDSQL_TABLE_NAME}' AND indexname LIKE '%_index'"))

[{'schemaname': 'public',
  'tablename': 'retrieval-cloudsql-postgresql',
  'indexname': 'ivfflat_index',
  'tablespace': None,
  'indexdef': 'CREATE INDEX ivfflat_index ON public."retrieval-cloudsql-postgresql" USING ivfflat (embedding vector_ip_ops) WITH (lists=\'100\')'},
 {'schemaname': 'public',
  'tablename': 'retrieval-cloudsql-postgresql',
  'indexname': 'hnsw_index',
  'tablespace': None,
  'indexdef': 'CREATE INDEX hnsw_index ON public."retrieval-cloudsql-postgresql" USING hnsw (embedding vector_ip_ops) WITH (m=\'10\', ef_construction=\'40\')'}]

Use distance measure, dot product with `<#>`, with the index:

In [118]:
run_query(sqlalchemy.text(f"""
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'dot_product': -0.7099841833114624},
 {'chunk_id': 'freddie_part_4_c509', 'dot_product': -0.680526077747345},
 {'chunk_id': 'freddie_part_4_c510', 'dot_product': -0.6753296852111816},
 {'chunk_id': 'fannie_part_0_c353', 'dot_product': -0.6723706722259521},
 {'chunk_id': 'fannie_part_0_c326', 'dot_product': -0.66834956407547}]

Use `EXPLAIN ANALYZE` to understand the query execution.  Note which index was used:

In [119]:
result = run_query(sqlalchemy.text(f"""
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <#> '{question_embedding}' AS dot_product
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY dot_product
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=100.38..103.00 rows=5 width=27) (actual time=0.976..1.008 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Index Scan using hnsw_index on "retrieval-cloudsql-postgresql"  (cost=100.38..4845.38 rows=9040 width=27) (actual time=0.974..1.005 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.132 ms'},
 {'QUERY PLAN': 'Execution Time: 1.039 ms'}]

#### Conditions Where Indexes Are Ignored - Forced Brute Force

There are conditions where the optimizer might not use an availabe index and instead do a full row scan for matches - brute force.

Using a different distance measure than was specified when building the index.  Here the cosine similarity is requested instead of dot product:

In [136]:
run_query(sqlalchemy.text(f"""
    SELECT chunk_id, embedding <=> '{question_embedding}' AS cosine_similarity
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY cosine_similarity
    LIMIT 5
"""))

[{'chunk_id': 'fannie_part_0_c352', 'cosine_similarity': 0.2899983636254655},
 {'chunk_id': 'freddie_part_4_c509', 'cosine_similarity': 0.31944424887732137},
 {'chunk_id': 'freddie_part_4_c510', 'cosine_similarity': 0.3246529458452222},
 {'chunk_id': 'fannie_part_0_c353', 'cosine_similarity': 0.32760391792511945},
 {'chunk_id': 'fannie_part_0_c326', 'cosine_similarity': 0.33164633285935186}]

Use `EXPLAIN ANALYZE` to understand the query execution.  Note that neither index was used:

In [135]:
result = run_query(sqlalchemy.text(f"""
EXPLAIN ANALYZE
    SELECT chunk_id, embedding <=> '{question_embedding}' AS cosine_similarity
    FROM \"{CLOUDSQL_TABLE_NAME}\"
    ORDER BY cosine_similarity
    LIMIT 5
"""))
result[0:2] + result[-2:]

[{'QUERY PLAN': 'Limit  (cost=1421.15..1421.16 rows=5 width=27) (actual time=38.639..38.643 rows=5 loops=1)'},
 {'QUERY PLAN': '  ->  Sort  (cost=1421.15..1443.75 rows=9040 width=27) (actual time=38.638..38.640 rows=5 loops=1)'},
 {'QUERY PLAN': 'Planning Time: 0.139 ms'},
 {'QUERY PLAN': 'Execution Time: 38.731 ms'}]

---
## Retrieval Augmented Generation (RAG)

Build a simple retrieval augmented generation process that enhances a query by retrieving context.  This is done here by constructing three functions for the stages:
- `retrieve` - a function that uses an embedding to search for matching context parts, pieces of texts
    - this uses the system built earlier in this workflow!
- `augment` - prepare chunks into a prompt
- `generate` - make the llm request with the augmented prompt

A final function is used to execute the workflow of rag:
- `rag` - a function that receives the query an orchestrates the workflow through `retrieve` > `augment` > `generate`

### Clients

In [32]:
embedder = vertexai.language_models.TextEmbeddingModel.from_pretrained('text-embedding-004')
llm = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")

### Retrieve Function

In [33]:
def retrieve_cloudsql_postgresql(query_embedding, n_matches = 5):
    
    matches = run_query(
        sqlalchemy.text(
            f"""
                SELECT chunk_id, content
                FROM \"{CLOUDSQL_TABLE_NAME}\"
                ORDER BY embedding <#> '{question_embedding}'
                LIMIT {n_matches}
            """)
    )
    
    return matches

### Augment Function

In [34]:
def augment(matches):

    prompt = ''
    for m, match in enumerate(matches):
        prompt += f"Context {m+1}:\n{match['content']}\n\n"
    prompt += f'Answer the following question using the provided contexts:\n'

    return prompt

### Generate Function

In [35]:
def generate(prompt):

    result = llm.generate_content(prompt)

    return result

### RAG Function

In [130]:
def rag(query):
    
    query_embedding = embedder.get_embeddings([query])[0].values
    matches = retrieve_cloudsql_postgresql(query_embedding)
    prompt = augment(matches) + query
    result = generate(prompt)
    
    return result.text

### Example In Use

In [131]:
question

'Does a lender have to perform servicing functions directly?'

In [132]:
print(rag(question))

No, a lender does not have to perform servicing functions directly.  Context 1 explicitly states that a lender may use other organizations to perform some or all of its servicing functions, referring to these arrangements as "subservicing."  The context also details situations that are *not* considered subservicing.  Further, Context 4 elaborates on the requirements and conditions under which a servicer (the "master servicer") may use a subservicer.



---
### Profiling Performance

Profile the timing of each step in the RAG function for sequential calls. The environment choosen for this workflow is a minimal testing enviornment so load testing (simoultaneous requests) would not be helpful.

In [37]:
profile = []

In [38]:
def rag(query, profile = profile):
    
    timings = {}
    start_time = time.time()
    
    
    # 1. Get embeddings
    embedding_start = time.time()
    query_embedding = embedder.get_embeddings([query])[0].values
    timings['embedding'] = time.time() - embedding_start

    # 2. Retrieve from Bigtable
    retrieval_start = time.time()
    matches = retrieve_cloudsql_postgresql(query_embedding)
    timings['retrieve_cloudsql_postgresql'] = time.time() - retrieval_start

    # 3. Augment the prompt
    augment_start = time.time()
    prompt = augment(matches) + query
    timings['augment'] = time.time() - augment_start

    # 4. Generate text
    generate_start = time.time()
    result = generate(prompt)
    timings['generate'] = time.time() - generate_start

    total_time = time.time() - start_time
    timings['total'] = total_time
    
    profile.append(timings)
    
    return result.text

In [39]:
print(rag(question))

No, a lender does not have to perform servicing functions directly.  Context 1 explicitly states that a lender "may use other organizations to perform some or all of its servicing functions," referring to this as "subservicing."  The contexts also detail the requirements and regulations surrounding such arrangements, including the roles of master servicers and subservicers.



In [40]:
profile

[{'embedding': 0.08710455894470215,
  'retrieve_cloudsql_postgresql': 0.8231306076049805,
  'augment': 2.47955322265625e-05,
  'generate': 0.6950137615203857,
  'total': 1.6052799224853516}]

In [41]:
for i in range(100):
    response = rag(question)

### Report From Profile

In [42]:
all_timings = {}
for timings in profile:
    for key, value in timings.items():
        if key not in all_timings:
            all_timings[key] = []
        all_timings[key].append(value)

In [43]:
for key, values in all_timings.items():
    arr = np.array(values)
    print(f"Statistics for '{key}':")
    print(f"  Min: {np.min(arr):.4f} seconds")
    print(f"  Max: {np.max(arr):.4f} seconds")
    print(f"  Mean: {np.mean(arr):.4f} seconds")
    print(f"  Median: {np.median(arr):.4f} seconds")
    print(f"  Std Dev: {np.std(arr):.4f} seconds")
    print(f"  P95: {np.percentile(arr, 95):.4f} seconds")
    print(f"  P99: {np.percentile(arr, 99):.4f} seconds")
    print("")

Statistics for 'embedding':
  Min: 0.0467 seconds
  Max: 0.0957 seconds
  Mean: 0.0558 seconds
  Median: 0.0537 seconds
  Std Dev: 0.0084 seconds
  P95: 0.0701 seconds
  P99: 0.0877 seconds

Statistics for 'retrieve_cloudsql_postgresql':
  Min: 0.0133 seconds
  Max: 0.8231 seconds
  Mean: 0.0229 seconds
  Median: 0.0147 seconds
  Std Dev: 0.0800 seconds
  P95: 0.0172 seconds
  P99: 0.0198 seconds

Statistics for 'augment':
  Min: 0.0000 seconds
  Max: 0.0001 seconds
  Mean: 0.0000 seconds
  Median: 0.0000 seconds
  Std Dev: 0.0000 seconds
  P95: 0.0001 seconds
  P99: 0.0001 seconds

Statistics for 'generate':
  Min: 0.5424 seconds
  Max: 1.2408 seconds
  Mean: 0.7252 seconds
  Median: 0.7044 seconds
  Std Dev: 0.1155 seconds
  P95: 0.9484 seconds
  P99: 1.0491 seconds

Statistics for 'total':
  Min: 0.6057 seconds
  Max: 1.6053 seconds
  Mean: 0.8040 seconds
  Median: 0.7731 seconds
  Std Dev: 0.1412 seconds
  P95: 1.0331 seconds
  P99: 1.3089 seconds



## Remove Resources

In [42]:
# can't drop the database of an active connection, switch connection to mysql (default) database

#sync_pool.dispose()
#sync_connector.close()
#sync_connector = google.cloud.sql.connector.Connector()
#sync_pool = get_sync_pool(sync_connector, 'postgres')

#await async_pool.dispose()
#await async_connector.close_async()
#async_connector = await google.cloud.sql.connector.create_async_connector()
#async_pool = await get_async_pool(async_connector, 'postgres')

#query = sqlalchemy.text(f"DROP DATABASE IF EXISTS \"{CLOUDSQL_DATABASE_NAME}\"")
#run_query(query)

In [107]:
#user_delete = !gcloud sql users delete $CLOUDSQL_USER --instance=$CLOUDSQL_INSTANCE_NAME --quiet

In [100]:
#instance_delete = !gcloud sql instances delete $CLOUDSQL_INSTANCE_NAME --quiet --format=json